In [1]:
#  Ebnable HTML/CSS 
from IPython.core.display import HTML
HTML("<link href='https://fonts.googleapis.com/css?family=Passion+One' rel='stylesheet' type='text/css'><style>div.attn { font-family: 'Helvetica Neue'; font-size: 30px; line-height: 40px; color: #FFFFFF; text-align: center; margin: 30px 0; border-width: 10px 0; border-style: solid; border-color: #5AAAAA; padding: 30px 0; background-color: #DDDDFF; }hr { border: 0; background-color: #ffffff; border-top: 1px solid black; }hr.major { border-top: 10px solid #5AAA5A; }hr.minor { border: none; background-color: #ffffff; border-top: 5px dotted #CC3333; }div.bubble { width: 65%; padding: 20px; background: #DDDDDD; border-radius: 15px; margin: 0 auto; font-style: italic; color: #f00; }em { color: #AAA; }div.c1{visibility:hidden;margin:0;height:0;}div.note{color:red;}</style>")

___
Enter Team Member Names here (*double click to edit*):

- Name 1:
- Name 2:
- Name 3:

___

# In Class Assignment Two
In the following assignment you will be asked to fill in python code and derivations for a number of different problems. Please read all instructions carefully and turn in the rendered notebook (or HTML of the rendered notebook)  before the end of class (or right after class). The initial portion of this notebook is given before class and the remainder is given during class. Please answer the initial questions before class, to the best of your ability. Once class has started you may rework your answers as a team for the initial part of the assignment. 

<a id="top"></a>
## Contents
* <a href="#Loading">Loading the Data</a>
* <a href="#svm">Linear SVMs</a>
* <a href="#svm_using">Using Linear SVMs</a>
* <a href="#nonlinear">Non-linear SVMs</a>

________________________________________________________________________________________________________

<a id="Loading"></a>
<a href="#top">Back to Top</a>
## Loading the Data
Please run the following code to read in the "olivetti faces" dataset from sklearn's data loading module. 

This will load the data into the variable `ds`. `ds` is a `bunch` object with fields like `ds.data` and `ds.target`. The field `ds.data` is a numpy matrix of the continuous features in the dataset. **The object is not a pandas dataframe. It is a numpy matrix.** Each row is a set of observed instances, each column is a different feature. It also has a field called `ds.target` that is an integer value we are trying to predict (i.e., a specific integer represents a specific person). Each entry in `ds.target` is a label for each row of the `ds.data` matrix. 

In [2]:
# fetch the images for the dataset
# this will take a long time the first run because it needs to download
# after the first time, the dataset will be save to your disk (in sklearn package somewhere) 
# if this does not run, you may need additional libraries installed on your system (install at your own risk!!)
from sklearn.datasets import fetch_lfw_people

lfw_people = fetch_lfw_people(min_faces_per_person=20, resize=None)

In [3]:
# get some of the specifics of the dataset
X = lfw_people.data
y = lfw_people.target
names = lfw_people.target_names

n_samples, n_features = X.shape
_, h, w = lfw_people.images.shape
n_classes = len(names)

print("n_samples: {}".format(n_samples))
print("n_features: {}".format(n_features))
print("n_classes: {}".format(n_classes))
print("Original Image Sizes {} by {}".format(h,w))
print (125*94) # the size of the images are the size of the feature vectors

n_samples: 3023
n_features: 11750
n_classes: 62
Original Image Sizes 125 by 94
11750


**Question 1:** For the faces dataset, describe what the data represents? That is, what is each column? What is each row? What do the unique class values represent?



*Enter you answer here (double click)*

This data represents ...


___

<a id="svm"></a>
<a href="#top">Back to Top</a>
## Linear Support Vector Machines

**Question 2:** If we were to train a linear Support Vector Machine (SVM) upon the faces data, how many parameters would need to be optimized in the model? That is, how many coefficients would need to be calculated?




*Enter you answer here (double click)*

Assuming we use one-versus all to train each binary SVM, we need one weight for each feature and class combination, 11750x62=728500, plus we need a parameter for each intercept (or bias)=728562


___

In [4]:
# Enter any scratchwork or calculations here

print('With all features: ', 11750*62+62)

With all features:  728562


**Question 3:** 
- **Part A:** Given the number of parameters calculated above, would you expect the model to train quickly using **batch optimization techniques**? Why or why not? 
- **Part B:** Is there a way to reduce training time?
- **Part C:** If we transformed the X data using principle components analysis (PCA) with 100 components, how many parameters would we need to find for a linear Support Vector Machine (SVM)?

*Enter you answer here (double click)*


A. We would not expect a quick response. With so many parameters, we need a massively parallel architecture to get speedup from the one-versus-all (62 parallel trainings). Also, with so many weights, each training will take a large amount of time. To speed up the calculations, we need to use something other than batch gradient descent (like stochastic gradient descent or mini-batch gradient descent). 

B. To reduce training time we could (1) sub-sample the dataset, (2) reduce the number of features (perhaps using PCA), and/or we could use a stochastic gradient descent approach to speed up convergence. 

___

In [5]:
# Enter any scratchwork or calculations here

print('Part C. With 100 features: ', 100*62+62)

Part C. With 100 features:  6262


___
<a id="svm_using"></a>
<a href="#top">Back to Top</a>

# Using Linear SVMs

**Exercise 1:** Use the block of code below to check if the number of parameters you calculated is equal to the number of parameters returned by `sklearn`'s implementation of the Linear SVM. **Was your calculation correct? If different, can you think of a reason why the parameters would not match?**

In [6]:
from sklearn.svm import LinearSVC
from sklearn.decomposition import RandomizedPCA

n_components = 100
pca = RandomizedPCA(n_components=n_components)
Xpca = pca.fit_transform(X)

clf = LinearSVC()
clf.fit(Xpca,y)

LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0)

In [7]:
#===================================================================
# Enter you code below to calculate the number of parameters in the model 

print(clf.coef_.shape[0]*clf.coef_.shape[1]+clf.intercept_.shape[0])

#===================================================================

6262


___
**Exercise 2:** Use the starter code below to calculate two quantities: 
- **Part A.:** The overall accuracy of the trained linear svm on the training set
- **Part B.:** The *mean, standard deviation, maximum, and minimum* of the **accuracy per class** on the training set

You might be interested in the following documentation of the confusion matrix calculated by `scikit-learn`:
- http://scikit-learn.org/stable/auto_examples/model_selection/plot_confusion_matrix.html

And an example matrix returned by the confusion matrix function:
<img src="http://scikit-learn.org/stable/_images/plot_confusion_matrix_001.png",width=400,height=400>

In [8]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
import numpy as np

yhat = clf.predict(Xpca)

#===================================================
# Enter your code below

print('Overall Accuracy is ',accuracy_score(y,yhat))

conf = confusion_matrix(y,yhat)
norm_conf = conf/conf.sum(axis=1)[:, np.newaxis]
class_acc = np.diag(norm_conf)
print('The class accuracy is ',np.mean(class_acc), '+-', np.std(class_acc),end='')
print(' (',np.min(class_acc), np.max(class_acc),')')

#===================================================

Overall Accuracy is  0.871319880913
The class accuracy is  0.885397744858 +- 0.128513161047 ( 0.4 1.0 )


___
<a id="nonlinear"></a>
<a href="#top">Back to Top</a>

# Non-linear SVMs
Now let's explore the use of non-linear svms. More explicitly, using different kernels. Take a look at the example training and testing code below for the non-linear SVM. All parameters are left as default, except we change the kernel to be `rbf`. Run the block of code below.



In [9]:
from sklearn.svm import SVC

clf = SVC(kernel='rbf')
clf.fit(Xpca,y)
yhat = clf.predict(Xpca)
print('Overall Accuracy is ',accuracy_score(y,yhat))

Overall Accuracy is  0.93648693351


___
**Exercise 3:** Use the starter code from above to calculate the accuracy for three different non-linear SVM kernels. That is, repeat the code above for different `kernel` parameters. ** Which kernel is most accurate with the default parameters?**

You might be interested in the documentation of the scikit-learn SVM implementation, available here:
- http://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html

In [10]:
#===================================================
# Enter your code below

for kern in ['rbf','poly','sigmoid']: # note that a linear kernel is also highly accurate
    clf = SVC(kernel=kern) 
    clf.fit(Xpca,y)
    yhat = clf.predict(Xpca)
    print('Overall Accuracy is ', kern, accuracy_score(y,yhat))    

#===================================================

Overall Accuracy is  linear 0.999007608336
Overall Accuracy is  rbf 0.93648693351
Overall Accuracy is  poly 0.975190208402
Overall Accuracy is  sigmoid 0.175322527291


*Enter you answer here (double click)*

The most accurate kernel is ... the polynomial kernel. Despite the linear kernel performing well.


___
**Exercise 4:** Choose the **most accurate kernel** and manipulate the settings for `gamma` to make the classification more accurate. 
- **Part A:** How accurate can you make it? 
- **Part B:** Would you expect the results to generalize well? Why or why not?

In [10]:
#===================================================
# Enter your code below

kern = 'poly'

for g in np.logspace(-2,1,10):
    yhat = SVC(kernel=kern, gamma=g).fit(Xpca,y).predict(Xpca)
    print('Overall Accuracy is for g=', g, 'is', 100*accuracy_score(y,yhat)) 

#===================================================

Overall Accuracy is for g= 0.01 is 97.5521005624
Overall Accuracy is for g= 0.0215443469003 is 100.0
Overall Accuracy is for g= 0.0464158883361 is 100.0
Overall Accuracy is for g= 0.1 is 100.0
Overall Accuracy is for g= 0.215443469003 is 100.0
Overall Accuracy is for g= 0.464158883361 is 100.0
Overall Accuracy is for g= 1.0 is 100.0
Overall Accuracy is for g= 2.15443469003 is 100.0
Overall Accuracy is for g= 4.64158883361 is 100.0
Overall Accuracy is for g= 10.0 is 100.0


*Enter you answer here (double click)*

The highest accuracy we could achieve was ... 100% accuracy with gamma around 0.02 and higher.

We (would/would not) expect this to generalize because .... We would not expect this to generalize. Because we are training and testing on the exact same data, we are likely overfit. This is a common problem with SVM kernel methods --- they can fit about any complex relationship. However, we are using regularization in the SVM (by default in sklearn its using L2 regularization), so you can also make an argument based on regularization helping us to generalize.



___
**Final Question:** Using the most accurate non-linear SVM you found in the previous question, how many parameter coefficients does the trained model contain?

*Enter you answer here (double click)*

This is a trick question. Kernel SVMs do not actually find the coefficients for the higher dimensional space. We only need to store the support vectors found. 

However, scikit-learn does store the $\alpha$ values that correspond to the support vectors for each class, so we can make an argument that these are the stored parameter coefficients. The number of them is calculated below, along with the intercepts of the bias terms. It is also fine to say that this question has no correct answer---but you need to validate that by saying non-linear kernels don't store the raw parameters.

In [23]:
#===================================================
# Enter any scratchwork calculations you need below

clf = SVC(kernel='poly', gamma=0.5)
clf.fit(Xpca,y)

print(clf.intercept_.shape[0], 'bias terms')
print(clf.dual_coef_.shape) # the alpha values, not neccessarily parameters, just used to index the support vectors
print(np.sum(clf.n_support_)) # sum up the total number of support vectors in each class!

print('Total number of stored parameters is:', np.sum(clf.n_support_)+clf.intercept_.shape[0])

1891
(61, 2938)
2938
Total number of stored parameters is: 4829


________________________________________________________________________________________________________

That's all! Please **save (make sure you saved!!!) and upload your rendered notebook** and please include **team member names** in the notebook submission.